In [19]:
#import warnings
#warnings.filterwarnings("ignore", message="numpy.dtype size changed")
#warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
import pandas as pd
import numpy as np
import os
import sys
from IPython.display import display
import time
from datetime import datetime
from features import *
data_loc = '../data/onepunch/'

In [20]:
def load_data():
    print("Loading Data... ")
    data = {}
    for file_name, column_names in file_column_mappings.items():
        nice_name = file_name_name_mapping.get(file_name, None)
        if nice_name:
            print("Loading %s"%(nice_name))
            file_name = data_loc+file_name
            data[nice_name] = pd.read_csv(file_name,
                                          sep='|',
                                          index_col=False, 
                                          names=column_names
                                        )
    print("Data Loaded!")
    return data

raw_data_dic = load_data()
raw_data = raw_data_dic['puborders']

Loading Data... 
Loading puborders
Data Loaded!


In [21]:
#smart way of counting null values.
null_value_status=raw_data.isnull().sum(axis=0)
null_value_status[null_value_status != 0]

order_id                                                    1
order_state                                                24
order_created_at                                           27
order_updated_at                                          212
original_order_id                                       30325
order_complexity                                         5274
order_image_score                                         271
order_image_score_name                                     27
priority                                                   27
order_deliverable                                          27
order_most_recent_failure_reason                        54853
site_id                                                    47
order_local                                               183
slayer_job_id                                           32408
order_site_name                                            27
sneaky                                                     27
order_sk

In [18]:
filtered_data = raw_data.loc[(raw_data['order_state'] == 'complete') 
                             & (raw_data['sneaky'] == 'f') 
                             &(raw_data['order_most_recent_failure_reason'].isna())]

selected_data = filtered_data[['order_id','order_created_at','order_complexity','order_image_score','priority'
                              ,'order_deliverable','site_id','order_skip_markups','number_of_stories',
                               'complexity_attributes_total_roof_facets_above_4_sqft',
                               'complexity_attributes_total_roof_facets','complexity_attributes_walls_square_footage',
                              'complexity_attributes_is_mfr','slayer_attributes_user_first_capture',
                               'poser_solve_status','visible_on_ortho_map','poser_solved_image_count',
                              'number_of_images','average_image_score','turn_around_time_first_completion']]
#display(selected_data)
selected_data["order_created_at"] = pd.to_datetime(selected_data["order_created_at"], utc=True, errors='coerce')
selected_data = selected_data[selected_data["order_created_at"].notnull()]
print(len(selected_data))

def convert_complexity(x):
    if x == 'Simple':
        return 0.0
    elif x == 'Average':
        return 1.0
    elif x == 'Complex Residential':
        return 2.0
    elif x == 'Custom':
        return 3.0
    else:
        return 4.0
def convert_deliverable(x):
    if x == 'roof':
        return 0.0
    elif x == 'complete':
        return 1.0
    else:
        return 2.0
    
def convert_true_false(x):
    if x=='t' or x=='true':
        return 1.0
    else:
        return 0.0
    
def convert_poser_output(x):
    if x == 'Success':
        return 1.0
    else:
        return 0.0


selected_data['order_complexity']=selected_data['order_complexity'].apply(convert_complexity)
selected_data['order_deliverable']=selected_data['order_deliverable'].apply(convert_deliverable)
selected_data['order_skip_markups']=selected_data['order_skip_markups'].apply(convert_true_false)
selected_data['complexity_attributes_is_mfr']=selected_data['complexity_attributes_is_mfr'].apply(convert_true_false)
selected_data['slayer_attributes_user_first_capture']=selected_data['slayer_attributes_user_first_capture'].apply(convert_true_false)
selected_data['visible_on_ortho_map']=selected_data['visible_on_ortho_map'].apply(convert_true_false)
selected_data['poser_solve_status']=selected_data['poser_solve_status'].apply(convert_poser_output)


selected_data['order_id'] = selected_data['order_id'].apply(lambda x: int(x))
selected_data['order_created_at_time'] = selected_data['order_created_at'].apply(lambda x:(x.hour + x.minute/60.0 )/24.0 )
selected_data['order_created_at'] = selected_data['order_created_at'].apply(lambda x: (x.dayofweek))

cols = selected_data.columns.tolist()
new_cols = cols[:-2]+cols[-1:]+cols[-2:-1]
selected_data = selected_data[new_cols]

print(len(selected_data))
#selected_data.dropna(inplace = True)
selected_data = selected_data.fillna(0)
print(len(selected_data))
display(selected_data)

24055
24055
24055


/home/tommy/myenv/lib/python2.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


,order_id,order_created_at,order_complexity,order_image_score,priority,order_deliverable,site_id,order_skip_markups,number_of_stories,complexity_attributes_total_roof_facets_above_4_sqft,...,complexity_attributes_walls_square_footage,complexity_attributes_is_mfr,slayer_attributes_user_first_capture,poser_solve_status,visible_on_ortho_map,poser_solved_image_count,number_of_images,average_image_score,order_created_at_time,turn_around_time_first_completion
0,745942,6,1.0,2.0,4.0,1.0,28.0,1.0,2,8,...,2041,0.0,0.0,0.0,1.0,0.0,16.0,1.0,0.082639,1639.659283
1,746286,6,0.0,3.0,4.0,1.0,29.0,1.0,2,3,...,1630,0.0,0.0,0.0,1.0,0.0,8.0,2.0,0.775000,84.623484
7,747842,0,1.0,1.0,1.0,1.0,28.0,1.0,2,10,...,1885,0.0,0.0,0.0,1.0,0.0,8.0,2.0,0.596528,88.370153
8,747990,0,0.0,2.0,4.0,1.0,28.0,0.0,1,2,...,921,0.0,0.0,1.0,0.0,3.0,8.0,2.0,0.656944,124.217999
9,748413,0,1.0,3.0,8.0,1.0,28.0,1.0,2,6,...,2086,0.0,0.0,0.0,1.0,0.0,14.0,2.0,0.819444,675.156139
14,750071,1,0.0,3.0,4.0,1.0,28.0,1.0,2,6,...,2023,0.0,0.0,0.0,1.0,0.0,12.0,1.0,0.631250,88.042513
15,750139,1,0.0,2.0,4.0,0.0,29.0,0.0,1,7,...,0,0.0,0.0,1.0,0.0,6.0,8.0,2.0,0.654861,80.955529
16,750298,1,0.0,3.0,9.0,1.0,28.0,1.0,2,5,...,2572,0.0,1.0,0.0,1.0,0.0,8.0,1.0,0.706250,920.571894
17,751045,1,2.0,3.0,4.0,1.0,28.0,1.0,2,21,...,1936,0.0,0.0,0.0,1.0,0.0,8.0,2.0,0.931944,458.880273
22,752471,2,1.0,3.0,4.0,1.0,28.0,1.0,2,13,...,2833,0.0,0.0,0.0,1.0,0.0,16.0,1.0,0.602778,128.329332


In [17]:
npsave = selected_data.values[:, :].astype(np.float32)
np.save('./big.npy', npsave)
n = np.load('./big.npy')

In [ ]:
#print(raw_data.head(5))
header=list(raw_data)
print('index','name','example')
for idx, name in enumerate(header):
    print(idx,name,raw_data.iloc[0,idx])

In [ ]:
failed_column = raw_data['order_most_recent_failure_reason']
print(failed_column.shape)
for i,s in enumerate(failed_column):
    if(i<10):
        print(s)
nan_failed_column = raw_data.loc[(raw_data['order_most_recent_failure_reason'].isna())]
print(nan_failed_column.shape)
failed_column = nan_failed_column['order_most_recent_failure_reason']
print(failed_column.shape)
for i,s in enumerate(failed_column):
    if(i<10):
        print(s)

In [ ]:
#filter data with uncessary information

filtered_data = raw_data.loc[(raw_data['order_state'] == 'complete') 
                             & (raw_data['sneaky'] == 'f') 
                             &(raw_data['order_most_recent_failure_reason'].isna())]
print(filtered_data.shape)

In [ ]:
#list(filtered_data)
selected_data = filtered_data[['order_id','order_created_at','order_complexity','order_image_score','priority'
                              ,'order_deliverable','site_id','order_skip_markups','number_of_stories',
                               'complexity_attributes_total_roof_facets_above_4_sqft',
                               'complexity_attributes_total_roof_facets','complexity_attributes_walls_square_footage',
                              'complexity_attributes_is_mfr','slayer_attributes_user_first_capture',
                               'poser_solve_status','visible_on_ortho_map','poser_solved_image_count',
                              'number_of_images','average_image_score','turn_around_time_first_completion']]
display(selected_data)

In [ ]:
def convert_time(date_string):
    datetime_object = datetime.strptime(date_string, "%Y-%m-%d %H:%M:%S.%f")
    return 0,(datetime_object.weekday() + (datetime_object.hour + datetime_object.minute / 60.0 ) / 24.0 ) / 7.0,\
    0,0,0,0
  
def convert_time_(date_string):
    datetime_object = datetime.strptime(date_string, "%Y-%m-%d %H:%M:%S.%f")
    return 0,#datetime_object.weekday(),\
    (datetime_object.weekday() + (datetime_object.hour + datetime_object.minute / 60.0 ) / 24.0 ) / 7.0,\
    0,#datetime_object.month,\
    0,#datetime_object.day,\
    0,#datetime_object.hour,\
    0#datetime_object.minute
    
def convert_complexity(x):
    if x == 'Simple':
        return 0.0
    elif x == 'Average':
        return 1.0
    elif x == 'Complex Residential':
        return 2.0
    elif x == 'Custom':
        return 3.0
    else:
        return 4.0
def convert_deliverable(x):
    if x == 'roof':
        return 0.0
    elif x == 'complete':
        return 1.0
    else:
        return 2.0
    
def convert_true_false(x):
    if x=='t' or x=='true':
        return 1.0
    else:
        return 0.0
    
def convert_poser_output(x):
    if x == 'Success':
        return 1.0
    else:
        return 0.0
    
output_data = selected_data.drop(columns=['order_created_at'])
output_data[['weekday', 'combine','month','day','hour','minute']] \
= (selected_data['order_created_at'].apply(convert_time)).apply(pd.Series)

output_data['order_complexity']=selected_data['order_complexity'].apply(convert_complexity)
output_data['order_deliverable']=selected_data['order_deliverable'].apply(convert_deliverable)
output_data['order_skip_markups']=selected_data['order_skip_markups'].apply(convert_true_false)
output_data['complexity_attributes_is_mfr']=selected_data['complexity_attributes_is_mfr'].apply(convert_true_false)
output_data['slayer_attributes_user_first_capture']=selected_data['slayer_attributes_user_first_capture'].apply(convert_true_false)
output_data['visible_on_ortho_map']=selected_data['visible_on_ortho_map'].apply(convert_true_false)
output_data['poser_solve_status']=selected_data['poser_solve_status'].apply(convert_poser_output)
output_data = output_data.fillna(0)

cols = output_data.columns.tolist()
print(cols)
new_cols = cols[:-7]+cols[-6:]+cols[-7:-6]
output_data = output_data[new_cols]

In [ ]:
header=list(output_data)
print('index','name','example')
for idx, name in enumerate(header):
    print(idx,name,output_data.iloc[3,idx])

In [ ]:
display(output_data.head(50))

In [ ]:
store_np_mat = output_data.values
#store_np_mat=pd.to_numeric(store_np_mat, errors='coerce')

In [ ]:
#print(store_np_mat)
#store_np_mat[0][0] = 'haha'
store_np_mat = store_np_mat.astype(np.double)
print(store_np_mat[0][0])
print(pd.isnull(store_np_mat).any())
print(np.isfinite(store_np_mat).all())

In [ ]:
np.save('./examed_data.npy', store_np_mat)

In [ ]:
import sklearn
examed_data = np.load('./data/new_data_no_fail.npy')
#examed_data = np.load('./data/new_data_no_fail.npy')

In [ ]:
print(examed_data.shape)
print(np.isnan(examed_data).any())

In [ ]:
np.random.shuffle(examed_data)

In [ ]:
print(examed_data[2])

In [ ]:
split = 20000
dimension = 13

train_x = examed_data[:split,1:dimension]
print(train_x.shape)
train_y = examed_data[:split,-1]
print(train_y.shape)

test_x = examed_data[split:,1:dimension]
test_y = examed_data[split:,-1]
print(test_x.shape)
print(test_y.shape)

In [ ]:
print(train_x[0])

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

regr = RandomForestRegressor(max_depth=20,n_estimators=500,oob_score=True)
#regr = AdaBoostRegressor(DecisionTreeRegressor(max_depth=20),
#                          n_estimators=500)
regr.fit(train_x, train_y)

In [ ]:
print(regr.score(test_x,test_y))
print(regr.oob_score_)

In [ ]:
('index', 'name', 'example')

(1, 'order_complexity', 1.0)
(2, 'order_image_score', 2.0)
(3, 'priority', 4.0)
(4, 'order_deliverable', 1.0)
(5, 'site_id', 28.0)
(6, 'order_skip_markups', 1.0)
(7, 'number_of_stories', '2')
(8, 'complexity_attributes_total_roof_facets_above_4_sqft', '8')
(9, 'complexity_attributes_total_roof_facets', '8')
(10, 'complexity_attributes_walls_square_footage', '2041')
(11, 'complexity_attributes_is_mfr', 0.0)
(12, 'slayer_attributes_user_first_capture', 0.0)
(13, 'poser_solve_status', 0.0)
(14, 'visible_on_ortho_map', 1.0)
(15, 'poser_solved_image_count', 0.0)
(16, 'number_of_images', 16.0)
(17, 'average_image_score', 1.0)
(18, 'weekday', 6)
(19, 'year', 2018)
(20, 'month', 6)
(21, 'day', 10)
(22, 'hour', 1)
(23, 'minute', 59)

for idx, importance in enumerate(regr.feature_importances_):
    print(idx+1,importance*100)

In [ ]:
total = 0
number = len(test_x)
for i in range(number):
    prediction = regr.predict([test_x[i]])[0]
    truth = test_y[i]
    total += abs(prediction - truth)
    
total = total/number
print(total)